parsing data

In [9]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import json
import os
import gzip  # Importato per la decompressione manuale gzip
import brotli  # Importato per la decompressione manuale Brotli


class VirtualSportsCollector:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36',
            'Accept': 'application/json, text/plain, */*',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.6',
            'Origin': 'https://www.eurobet.it',
            'Referer': 'https://www.eurobet.it/',
            'X-EB-Accept-Language': 'it_IT',
            'X-EB-MarketId': '5',
            'X-EB-PlatformId': '1',
            'Connection': 'keep-alive',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-site'
        }
        self.base_url = "https://virtualservice.eurobet.it/virtual-winning-service/virtual-schedule/services/winningresult/68/17/{}"
        self.csv_filename = "virtual_matches_data.csv"
        self.excel_filename = "virtual_matches_data.xlsx"

    def create_match_id(self, row):
        """Crea un identificatore univoco per ogni partita."""
        date_val = str(row.get('date', ''))
        hour_val = str(row.get('hour', ''))
        home_team_val = str(row.get('home_team', ''))
        away_team_val = str(row.get('away_team', ''))
        return f"{date_val}_{hour_val}_{home_team_val}_{away_team_val}"

    def load_existing_data(self):
        """Carica i dati esistenti dal CSV, se esiste."""
        if os.path.exists(self.csv_filename):
            try:
                dtype_spec = {  # Specifica dtype per colonne potenzialmente problematiche
                    'odds_1': 'object', 'result': 'object',
                    'over_under_25': 'object', 'odds_over_under_25': 'object',
                    'goal_no_goal': 'object', 'odds_goal_no_goal': 'object',
                    'home_goals': 'Int64', 'away_goals': 'Int64'  # Usa Int64 per permettere NaN interi
                }
                df = pd.read_csv(self.csv_filename, dtype=dtype_spec)
                if 'datetime' in df.columns:
                    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
                # Rimuovi colonne completamente vuote che potrebbero essere state create da errori precedenti
                df.dropna(axis=1, how='all', inplace=True)
                return df
            except pd.errors.EmptyDataError:
                print(f"Il file CSV {self.csv_filename} è vuoto. Verrà creato un nuovo DataFrame.")
                return pd.DataFrame()
            except Exception as e:
                print(f"Errore durante il caricamento del file CSV {self.csv_filename}: {e}")
                return pd.DataFrame()
        return pd.DataFrame()

    def get_virtual_data(self, start_date, end_date):
        """Recupera i dati virtuali per l'intervallo di date specificato."""
        all_matches = []
        current_date = start_date

        while current_date <= end_date:
            date_str = current_date.strftime("%d-%m-%Y")
            url = self.base_url.format(date_str)
            print(f"Tentativo di recupero dati per {date_str} da URL: {url}")
            data = None
            response = None

            try:
                response = requests.get(url, headers=self.headers, timeout=25)
                response.raise_for_status()

                if not response.content:
                    print(f"Risposta vuota ricevuta per {date_str} (Status: {response.status_code}). URL: {url}")
                    time.sleep(1.5)
                    current_date += timedelta(days=1)
                    continue

                try:
                    data = response.json()
                except json.JSONDecodeError:
                    print(f"Errore di decodifica JSON standard per {date_str}. Status: {response.status_code}")
                    content_encoding = response.headers.get('Content-Encoding', '').lower()
                    print(f"Header Content-Encoding: {content_encoding if content_encoding else 'Non presente'}")

                    decompressed_successfully = False
                    if content_encoding == 'br':
                        print("Tentativo di decompressione Brotli manuale.")
                        try:
                            decompressed_content = brotli.decompress(response.content)
                            data = json.loads(decompressed_content.decode('utf-8'))
                            print(f"Contenuto Brotli per {date_str} decompresso e parsato manualmente.")
                            decompressed_successfully = True
                        except Exception as e_decompress:
                            print(f"Fallimento decompressione/parsing Brotli per {date_str}: {e_decompress}")

                    elif content_encoding == 'gzip' or response.content.startswith(b'\x1f\x8b\x08'):
                        print("Tentativo di decompressione Gzip manuale.")
                        try:
                            decompressed_content = gzip.decompress(response.content)
                            data = json.loads(decompressed_content.decode('utf-8'))
                            print(f"Contenuto Gzip per {date_str} decompresso e parsato manualmente.")
                            decompressed_successfully = True
                        except Exception as e_decompress:
                            print(f"Fallimento decompressione/parsing Gzip per {date_str}: {e_decompress}")

                    if not decompressed_successfully:
                        print(f"Decodifica JSON fallita per {date_str} anche dopo tentativi manuali (se applicabili).")
                        print(f"Contenuto grezzo (primi 200 byte): {response.content[:200]}...")
                        time.sleep(1.5)
                        current_date += timedelta(days=1)
                        continue

                # Elaborazione dei dati
                if data and 'result' in data and data['result'] is not None and 'groupDate' in data['result']:
                    for group in data['result']['groupDate']:
                        if 'events' in group and group['events'] is not None:
                            for event in group['events']:
                                try:
                                    # --- Inizio Logica di Parsing Team Names Migliorata ---
                                    parsed_home_team = None
                                    parsed_away_team = None

                                    event_desc_raw = event.get('eventDescription')

                                    if isinstance(event_desc_raw, str) and event_desc_raw.strip():
                                        cleaned_desc = event_desc_raw.strip()
                                        parts = cleaned_desc.split(' - ', 1)  # Divide al massimo una volta

                                        if parts[0]:  # Nome squadra casa
                                            parsed_home_team = parts[0].strip()
                                            if not parsed_home_team:  # Se solo spazi
                                                parsed_home_team = None

                                        if len(parts) > 1:  # Se il separatore " - " è stato trovato
                                            if parts[1]:  # Nome squadra ospite
                                                parsed_away_team = parts[1].strip()
                                                if not parsed_away_team:  # Se solo spazi o stringa vuota
                                                    parsed_away_team = None
                                            # else: parsed_away_team rimane None (es. "Team A - ")
                                        # else: parsed_away_team rimane None (es. "Team A")
                                    elif event_desc_raw is not None:  # Se non è stringa ma esiste (improbabile per descrizione)
                                        temp_desc = str(event_desc_raw).strip()
                                        if temp_desc:
                                            parsed_home_team = temp_desc
                                    # --- Fine Logica di Parsing Team Names Migliorata ---

                                    match_data = {
                                        'date': event.get('date'),
                                        'hour': event.get('hour'),
                                        'home_team': parsed_home_team,  # Usa il nome parsato
                                        'away_team': parsed_away_team,  # Usa il nome parsato
                                        'score': event.get('finalResult'),
                                        'home_goals': None,
                                        'away_goals': None,
                                        'datetime': None
                                    }

                                    if event.get('date') and event.get('hour'):
                                        try:
                                            match_data['datetime'] = pd.to_datetime(f"{event['date']} {event['hour']}",
                                                                                    format='%d-%m-%Y %H:%M:%S',
                                                                                    errors='coerce')
                                        except ValueError as ve:
                                            print(
                                                f"Errore formato data/ora: {event.get('date')} {event.get('hour')}: {ve}")

                                    if event.get('finalResult') and '-' in event['finalResult']:
                                        try:
                                            scores = event['finalResult'].split('-')
                                            match_data['home_goals'] = int(scores[0])
                                            match_data['away_goals'] = int(scores[1])
                                        except ValueError:
                                            print(
                                                f"Score non parsabile: {event['finalResult']} per {event.get('eventDescription')}")

                                    if 'oddGroup' in event and event['oddGroup'] is not None:
                                        for odd_group in event['oddGroup']:
                                            bet_abbr = odd_group.get('betDescriptionAbbr')
                                            odds_list = odd_group.get('odds')
                                            result_desc_list = odd_group.get('resultDescription')

                                            if not (odds_list and result_desc_list): continue

                                            if bet_abbr == '1X2':
                                                match_data['odds_1'] = odds_list[0] if odds_list else None
                                                match_data['result'] = result_desc_list[0] if result_desc_list else None
                                            elif bet_abbr == 'Under / Over 2.5':
                                                match_data['over_under_25'] = result_desc_list[
                                                    0] if result_desc_list else None
                                                match_data['odds_over_under_25'] = odds_list[0] if odds_list else None
                                            elif bet_abbr == 'U/O 2.5':
                                                # print(f"Nota: Trovato 'U/O 2.5' (invece di 'Under / Over 2.5') per {date_str}, evento {event.get('eventDescription')}")
                                                match_data['over_under_25'] = result_desc_list[
                                                    0] if result_desc_list else None
                                                match_data['odds_over_under_25'] = odds_list[0] if odds_list else None
                                            elif bet_abbr == 'Goal/No Goal':
                                                match_data['goal_no_goal'] = result_desc_list[
                                                    0] if result_desc_list else None
                                                match_data['odds_goal_no_goal'] = odds_list[0] if odds_list else None

                                    all_matches.append(match_data)
                                except Exception as e_inner:
                                    print(
                                        f"Errore elaborazione evento ({date_str}): {e_inner}. Evento: {json.dumps(event, indent=2)}")
                        else:
                            print(f"Nessun evento ('events') in groupDate per {date_str}.")
                elif data:
                    print(f"Struttura JSON inattesa per {date_str}. Dati (inizio): {str(data)[:200]}...")
                else:
                    print(f"Nessun dato JSON valido per {date_str} dopo tentativi di decompressione.")

            except requests.exceptions.HTTPError as http_err:
                print(f"Errore HTTP per {date_str}: {http_err} (Status: {response.status_code if response else 'N/A'})")
            except requests.exceptions.ConnectionError as conn_err:
                print(f"Errore di connessione per {date_str}: {conn_err}")
            except requests.exceptions.Timeout as timeout_err:
                print(f"Timeout per {date_str}: {timeout_err}")
            except Exception as e:
                print(f"Errore imprevisto recupero dati ({date_str}): {e}")

            time.sleep(1.5)
            current_date += timedelta(days=1)

        return pd.DataFrame(all_matches)

    def merge_and_save_data(self, new_data):
        """Unisci i nuovi dati con quelli esistenti, rimuovi sempre i duplicati e salva."""
        existing_data = self.load_existing_data()

        if not existing_data.empty and 'datetime' in existing_data.columns:
            existing_data['datetime'] = pd.to_datetime(existing_data['datetime'], errors='coerce')
        if not new_data.empty and 'datetime' in new_data.columns:
            new_data['datetime'] = pd.to_datetime(new_data['datetime'], errors='coerce')

        if new_data.empty:
            print("Nessun nuovo dato da unire. Processando e risalvando i dati esistenti (se presenti).")
            if existing_data.empty:
                print("Database esistente è vuoto. Nessun dato da salvare o processare.")
                pd.DataFrame().to_csv(self.csv_filename, index=False)
                pd.DataFrame().to_excel(self.excel_filename, index=False)
                print(f"Creati/aggiornati file vuoti: {self.csv_filename} e {self.excel_filename}")
                return pd.DataFrame()
            combined_data = existing_data
        else:
            print(f"Unendo {len(new_data)} nuove partite con {len(existing_data)} partite esistenti.")
            combined_data = pd.concat([existing_data, new_data], ignore_index=True)

        if combined_data.empty:
            print("Nessun dato (né esistente né nuovo) da processare.")
            return pd.DataFrame()

        print(f"Dati combinati prima della pulizia: {len(combined_data)} righe.")

        if 'datetime' in combined_data.columns:
            initial_rows = len(combined_data)
            combined_data.dropna(subset=['datetime'], inplace=True)
            if len(combined_data) < initial_rows:
                print(f"Rimosse {initial_rows - len(combined_data)} righe con datetime non valido.")

        combined_data['match_id'] = combined_data.apply(self.create_match_id, axis=1)

        combined_data = combined_data[combined_data['match_id'] != "___"]
        combined_data.dropna(subset=['match_id'], inplace=True)

        if combined_data.empty:
            print("Nessun dato valido rimasto dopo la pulizia iniziale di match_id.")
            pd.DataFrame().to_csv(self.csv_filename, index=False)
            pd.DataFrame().to_excel(self.excel_filename, index=False)
            return pd.DataFrame()

        print(f"Dati prima della deduplicazione: {len(combined_data)} righe.")
        if 'datetime' in combined_data.columns and not combined_data.empty:
            combined_data = combined_data.sort_values('datetime', ascending=False)
            combined_data = combined_data.drop_duplicates(subset=['match_id'], keep='first')
        elif not combined_data.empty:
            combined_data = combined_data.drop_duplicates(subset=['match_id'], keep='first')
        print(f"Dati dopo la deduplicazione: {len(combined_data)} righe.")

        if 'match_id' in combined_data.columns:
            combined_data = combined_data.drop('match_id', axis=1)

        try:
            combined_data.to_csv(self.csv_filename, index=False)
            combined_data.to_excel(self.excel_filename, index=False)
            print(
                f"Dati salvati con successo ({len(combined_data)} righe) in {self.csv_filename} e {self.excel_filename}")
        except Exception as e:
            print(f"Errore durante il salvataggio dei file: {e}")

        return combined_data

    def collect_data(self, days_to_fetch_count=1):
        """Metodo principale per raccogliere, elaborare e salvare i dati."""
        if not isinstance(days_to_fetch_count, int) or days_to_fetch_count < 0:
            print("Errore: days_to_fetch_count deve essere un intero non negativo (0 per oggi, >0 per giorni passati).")
            return

        today = datetime.now()
        start_date_of_collection, end_date_of_collection = None, None

        if days_to_fetch_count == 0:
            start_date_of_collection = today
            end_date_of_collection = today
            print(f"Richiesta dati per oggi: {today.strftime('%d-%m-%Y')}")
        else:
            end_date_of_collection = today - timedelta(days=1)
            start_date_of_collection = end_date_of_collection - timedelta(days=days_to_fetch_count - 1)
            if days_to_fetch_count == 1:
                print(f"Richiesta dati per ieri: {end_date_of_collection.strftime('%d-%m-%Y')}")
            else:
                print(
                    f"Richiesta dati dal {start_date_of_collection.strftime('%d-%m-%Y')} al {end_date_of_collection.strftime('%d-%m-%Y')}")

        new_data = pd.DataFrame()
        if start_date_of_collection <= end_date_of_collection:
            new_data = self.get_virtual_data(start_date_of_collection, end_date_of_collection)
        else:
            print(
                f"Intervallo date non valido: start {start_date_of_collection}, end {end_date_of_collection}. Salto recupero dati.")

        if not new_data.empty:
            print(f"Recuperate {len(new_data)} nuove partite dalla fonte.")
        else:
            print("Nessun nuovo dato raccolto dalla fonte.")

        final_data = self.merge_and_save_data(new_data)

        if final_data is not None and not final_data.empty:
            print(f"Database finale contiene {len(final_data)} partite.")
        elif final_data is not None and final_data.empty:
            print("Il database finale è vuoto.")
        else:
            print("Errore: final_data è None dopo merge_and_save_data.")


def main(days_to_fetch_count=1):
    """Funzione principale per avviare il collettore di dati."""
    collector = VirtualSportsCollector()
    collector.collect_data(days_to_fetch_count)


if __name__ == "__main__":
    GIORNI_DA_RECUPERARE = 0

    print(f"Avvio script. GIORNI_DA_RECUPERARE impostato a: {GIORNI_DA_RECUPERARE}")
    main(days_to_fetch_count=GIORNI_DA_RECUPERARE)
    print("Script terminato.")


Avvio script. GIORNI_DA_RECUPERARE impostato a: 0
Richiesta dati per oggi: 15-05-2025
Tentativo di recupero dati per 15-05-2025 da URL: https://virtualservice.eurobet.it/virtual-winning-service/virtual-schedule/services/winningresult/68/17/15-05-2025
Recuperate 28 nuove partite dalla fonte.
Unendo 28 nuove partite con 30891 partite esistenti.
Dati combinati prima della pulizia: 30919 righe.
Dati prima della deduplicazione: 30919 righe.
Dati dopo la deduplicazione: 30891 righe.
Dati salvati con successo (30891 righe) in virtual_matches_data.csv e virtual_matches_data.xlsx
Database finale contiene 30891 partite.
Script terminato.


lettura df

In [ ]:
data = pd.read_csv('virtual_matches_data.csv')

filtro per model data

In [5]:
data = data[['date', 'hour', 'home_team', 'away_team', 'home_goals', 'away_goals', 'result']]

In [6]:
data.to_csv('model_data.csv',index = False)

In [7]:
data.head(10)

,date,hour,home_team,away_team,home_goals,away_goals,result
0,15-05-2025,12:40:00,Messico,Serbia,2,2,X
1,15-05-2025,12:34:00,Italia,Spagna,0,1,2
2,15-05-2025,12:28:00,Croazia,Olanda,3,1,1
3,15-05-2025,12:22:00,Colombia,Messico,1,2,2
4,15-05-2025,12:16:00,Cile,Inghilterra,0,2,2
5,15-05-2025,12:10:00,Germania,Nigeria,0,1,2
6,15-05-2025,12:04:00,Francia,Inghilterra,0,0,X
7,15-05-2025,11:58:00,Brasile,Spagna,2,0,1
8,15-05-2025,11:52:00,Belgio,Svizzera,1,0,1
9,15-05-2025,11:46:00,Argentina,Turchia,0,3,2


Predizione Catboost

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Impostazioni per una migliore visualizzazione
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.4f}'.format) # Formattazione per i float

# Funzione helper per calcolare la lunghezza della striscia corrente di 'non-eventi'
# Un 'evento' (condizione negativa) resetta la striscia.
# Modificata per essere più generica nel nome, ma la logica è la stessa.
def _calculate_streak_ending_here(series_is_event_resets_streak):
    """
    Calcola la lunghezza della striscia corrente di 'non-eventi'.
    Un 'evento' (series_is_event_resets_streak == True) resetta la striscia a 0.
    Altrimenti, la striscia incrementa.
    """
    streaks = []
    current_streak = 0
    for is_reset_event in series_is_event_resets_streak:
        if is_reset_event: # Se l'evento che resetta la striscia si verifica
            current_streak = 0
        else: # L'evento non si verifica, quindi la striscia continua (o inizia se era 0)
            current_streak += 1
        streaks.append(current_streak)
    return pd.Series(streaks, index=series_is_event_resets_streak.index)

print("Ciao Andrea! Iniziamo l'analisi per la predizione della distanza tra pareggi usando CatBoost.")

# Caricamento dei dati
try:
    data = pd.read_csv('model_data.csv')
    print(f"Dati caricati con successo. Numero di righe: {len(data)}, Numero di colonne: {len(data.columns)}")
except FileNotFoundError:
    print("Errore: File 'model_data.csv' non trovato. Assicurati che sia nel percorso corretto.")
    exit()
except Exception as e:
    print(f"Errore durante il caricamento dei dati: {e}")
    exit()

# --- 1. Preparazione Dati Iniziale ---
print("\n--- Inizio Fase 1: Preparazione Dati Iniziale ---")

df = data.copy()

# Conversione 'date' e 'hour' in datetime
# Gestione del formato data: il file CSV originale ha i dati più recenti in alto.
# Ordiniamo esplicitamente per data/ora ascendente.
try:
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['hour'], format='%d/%m/%Y %H:%M', errors='coerce')
except ValueError: # Fallback per altri formati se necessario, o se il formato non è consistente
    print("Formato data/ora non riconosciuto come 'dd/mm/yyyy HH:MM'. Tentativo con infer_datetime_format.")
    try:
        df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['hour'], infer_datetime_format=True, errors='coerce')
    except Exception as e_dt:
        print(f"Errore finale nella conversione datetime: {e_dt}")
        df['datetime'] = pd.NaT # Imposta a Not a Time in caso di fallimento

if df['datetime'].isnull().any():
    print("Attenzione: Alcune date non sono state convertite correttamente in datetime.")
    print(df[df['datetime'].isnull()][['date', 'hour']].head())
    # Potresti voler gestire queste righe o uscire, a seconda della gravità
    # exit()

df.dropna(subset=['datetime'], inplace=True)
if df.empty:
    print("DataFrame vuoto dopo la rimozione di righe con datetime non validi.")
    exit()

# Ordinamento cronologico: ESSENZIALE per calcoli di strisce e split temporali
df = df.sort_values('datetime').reset_index(drop=True)
df['match_id'] = df.index # Utile per riferimenti futuri se necessario

# Definizione di base del pareggio
df['is_draw'] = (df['home_goals'] == df['away_goals']).astype(int)

# Feature: Global distance since last draw (NON-LEAKY)
# Questa feature calcola, per ogni partita, quanti incontri sono trascorsi dall'ultimo pareggio globale.
# Se la partita corrente è un pareggio, questa feature indica la lunghezza della striscia di non-pareggi
# che si è appena conclusa.
# Input per _calculate_streak_ending_here è df['is_draw'].
# Se is_draw è True (pareggio), la striscia di non-pareggi si interrompe (reset a 0).
# Se is_draw è False (non pareggio), la striscia di non-pareggi continua.
df['_global_current_non_draw_streak'] = _calculate_streak_ending_here(df['is_draw'])
# _global_current_non_draw_streak:
# Esempio: is_draw = [F, F, F, T, F, T] -> [0,0,0,1,0,1]
# _calc_streak :       [1, 2, 3, 0, 1, 0] (conta i non-pareggi, resetta a 0 sul pareggio)

# global_distance_since_last_draw è il valore di _global_current_non_draw_streak della partita precedente.
# Rappresenta il numero di non-pareggi consecutivi *prima* della partita corrente.
df['global_distance_since_last_draw'] = df['_global_current_non_draw_streak'].shift(1)
df['global_distance_since_last_draw'].fillna(0, inplace=True) # Per la prima partita, o se la precedente era un pareggio.
df.drop(columns=['_global_current_non_draw_streak'], inplace=True)

print("Feature 'global_distance_since_last_draw' calcolata.")
print(df[['datetime', 'is_draw', 'global_distance_since_last_draw']].head())
print("\n--- Fine Fase 1: Preparazione Dati Iniziale ---")


# --- 2. Estrazione e Preparazione della Sequenza di Intervalli tra Pareggi ---
print("\n--- Inizio Fase 2: Estrazione Sequenza Intervalli tra Pareggi ---")

# Filtriamo le partite che sono finite in pareggio
df_draw_events = df[df['is_draw'] == 1].copy()

# La colonna 'global_distance_since_last_draw' in df_draw_events ora rappresenta
# la lunghezza della striscia di non-pareggi che si è conclusa con quel pareggio.
# Questo è il nostro "intervallo tra pareggi".
df_draw_events.rename(columns={'global_distance_since_last_draw': 'draw_interval_length'}, inplace=True)

# Potremmo voler escludere intervalli di lunghezza 0 se il primo match del dataset è un pareggio,
# o se ci sono pareggi consecutivi (improbabile con questa definizione di 'global_distance_since_last_draw'
# che dovrebbe dare 0 per un pareggio immediatamente successivo a un altro).
# Per ora, li manteniamo, ma è un punto da considerare.
# df_draw_events = df_draw_events[df_draw_events['draw_interval_length'] > 0]

if df_draw_events.empty:
    print("Nessun evento di pareggio trovato nel dataset. Impossibile procedere.")
    exit()

print(f"Numero di eventi di pareggio identificati: {len(df_draw_events)}")
print("Primi eventi di pareggio e la lunghezza dell'intervallo corrispondente:")
print(df_draw_events[['datetime', 'draw_interval_length']].head())

# Creazione di features laggate per predire il prossimo intervallo
N_LAGS = 5 # Numero di intervalli passati da usare come features
print(f"\nCreazione di {N_LAGS} features laggate dagli intervalli tra pareggi...")

for i in range(1, N_LAGS + 1):
    df_draw_events[f'lag_{i}_interval'] = df_draw_events['draw_interval_length'].shift(i)

# Aggiunta di altre features basate sulla sequenza storica, se desiderato:
# Esempio: media mobile degli ultimi N_LAGS intervalli (escludendo il corrente)
# df_draw_events['moving_avg_interval'] = df_draw_events['draw_interval_length'].shift(1).rolling(window=N_LAGS, min_periods=1).mean()

# Rimuoviamo le righe con NaN generati dalla creazione dei lag (le prime N_LAGS righe)
df_draw_events.dropna(inplace=True)

if df_draw_events.empty:
    print(f"DataFrame degli eventi di pareggio vuoto dopo aver creato {N_LAGS} lag e rimosso i NaN.")
    print("Potrebbe essere necessario un dataset più grande o un numero inferiore di lag.")
    exit()

print(f"Dimensioni del dataset per il modello dopo la creazione dei lag: {df_draw_events.shape}")
print("Esempio di dati con features laggate:")
cols_to_show_lags = ['datetime', 'draw_interval_length'] + [f'lag_{i}_interval' for i in range(1, N_LAGS + 1)]
print(df_draw_events[cols_to_show_lags].head())

print("\n--- Fine Fase 2: Estrazione Sequenza Intervalli tra Pareggi ---")


# --- 3. Preparazione Dati per il Modello CatBoost ---
print("\n--- Inizio Fase 3: Preparazione Dati per CatBoost ---")
features_for_model = [f'lag_{i}_interval' for i in range(1, N_LAGS + 1)]
# if 'moving_avg_interval' in df_draw_events.columns:
#     features_for_model.append('moving_avg_interval')

target_variable = 'draw_interval_length'

X = df_draw_events[features_for_model]
y = df_draw_events[target_variable]

if len(X) < 50: # Valore arbitrario, ma un dataset molto piccolo è problematico
    print(f"Attenzione: dataset per il modello molto piccolo ({len(X)} campioni). Le performance potrebbero essere scarse.")

# Split temporale dei dati (essendo una serie storica)
# Manteniamo l'ordine cronologico
train_size_ratio, val_size_ratio = 0.7, 0.15 # 70% train, 15% validation, 15% test

train_idx = int(train_size_ratio * len(X))
val_idx = int((train_size_ratio + val_size_ratio) * len(X))

X_train, y_train = X.iloc[:train_idx], y.iloc[:train_idx]
X_val, y_val = X.iloc[train_idx:val_idx], y.iloc[train_idx:val_idx]
X_test, y_test = X.iloc[val_idx:], y.iloc[val_idx:]

print(f"Dimensioni dei set: Train={X_train.shape}, Validation={X_val.shape}, Test={X_test.shape}")

if X_train.empty or X_test.empty:
    print("Errore: Train o Test set è vuoto. Controllare la dimensione del dataset e le percentuali di split.")
    exit()
# Il validation set può essere vuoto se val_size_ratio è 0 o troppo piccolo.

print("\n--- Fine Fase 3: Preparazione Dati per CatBoost ---")


# --- 4. Sviluppo del Modello CatBoost Regressor ---
print("\n--- Inizio Fase 4: Sviluppo del Modello CatBoost ---")

model = CatBoostRegressor(
    iterations=1000,        # Numero di alberi
    learning_rate=0.05,
    depth=6,
    loss_function='RMSE',   # Metrica di loss comune per la regressione
    eval_metric='MAE',      # Metrica per l'early stopping e la valutazione intermedia
    random_seed=42,
    verbose=200,            # Stampa info ogni 200 iterazioni
    # early_stopping_rounds=50 # Attiva l'early stopping
)

print("Inizio addestramento del modello CatBoostRegressor...")
fit_params = {}
if not X_val.empty and not y_val.empty:
    fit_params['eval_set'] = (X_val, y_val)
    fit_params['early_stopping_rounds'] = 50
    print("Utilizzo di un set di validazione per l'early stopping.")
else:
    print("Nessun set di validazione fornito o è vuoto. Addestramento senza early stopping basato su eval_set.")

try:
    model.fit(X_train, y_train, **fit_params)
    print("Modello CatBoost addestrato con successo.")
    model_fitted_successfully = True
except Exception as e:
    print(f"Errore durante l'addestramento del modello CatBoost: {e}")
    model_fitted_successfully = False

print("\n--- Fine Fase 4: Sviluppo del Modello CatBoost ---")


# --- 5. Valutazione del Modello ---
if model_fitted_successfully and not X_test.empty:
    print("\n--- Inizio Fase 5: Valutazione del Modello ---")
    y_pred_test = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred_test)
    mae = mean_absolute_error(y_test, y_pred_test)
    r2 = r2_score(y_test, y_pred_test)
    rmse = np.sqrt(mse)

    print("\nValutazione sul Test Set:")
    print(f"  Mean Squared Error (MSE): {mse:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"  Mean Absolute Error (MAE): {mae:.4f}")
    print(f"  R-squared (R²): {r2:.4f}")

    # Plot Actual vs. Predicted
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred_test, alpha=0.6, edgecolors='w', linewidth=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2) # Linea y=x
    plt.xlabel("Valori Reali (Lunghezza Intervallo)")
    plt.ylabel("Valori Predetti (Lunghezza Intervallo)")
    plt.title("Confronto Valori Reali vs. Predetti sul Test Set")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Plot Feature Importance
    if hasattr(model, 'get_feature_importance'):
        feature_importances = model.get_feature_importance()
        importance_df = pd.DataFrame({
            'feature': features_for_model,
            'importance': feature_importances
        }).sort_values('importance', ascending=False)

        plt.figure(figsize=(10, max(6, len(features_for_model) * 0.5)))
        sns.barplot(x='importance', y='feature', data=importance_df)
        plt.title('Importanza delle Features (CatBoost)')
        plt.tight_layout()
        plt.show()
        print("\nImportanza delle features:\n", importance_df)

    print("\n--- Fine Fase 5: Valutazione del Modello ---")
elif not model_fitted_successfully:
    print("\n--- Fase 5: Valutazione del Modello SKIPPATA (addestramento fallito) ---")
else:
    print("\n--- Fase 5: Valutazione del Modello SKIPPATA (Test set vuoto) ---")


# --- 6. Predizione del Prossimo Intervallo (Esempio) ---
if model_fitted_successfully and len(y) > N_LAGS: # Assicurati di avere abbastanza dati storici
    print("\n--- Inizio Fase 6: Predizione del Prossimo Intervallo ---")

    # Prendiamo gli ultimi N_LAGS intervalli osservati dall'intero dataset df_draw_events
    # Questi sarebbero gli input per predire il successivo intervallo non ancora osservato.
    last_known_intervals_series = df_draw_events['draw_interval_length'].tail(N_LAGS)

    if len(last_known_intervals_series) == N_LAGS:
        # Prepara l'input per il modello. CatBoost si aspetta un DataFrame
        # con nomi di colonna corrispondenti a quelli usati durante l'addestramento.
        # I valori devono essere nell'ordine corretto: lag_1 è il più recente dei lag, lag_N è il più vecchio.
        # Quindi, se last_known_intervals_series è [I(t-N+1), ..., I(t-1), I(t)],
        # lag_1 = I(t), lag_2 = I(t-1), ..., lag_N = I(t-N+1)

        input_data_for_prediction = {}
        for i in range(N_LAGS):
            # lag_{i+1}_interval corrisponde a last_known_intervals_series.iloc[N_LAGS - 1 - i]
            input_data_for_prediction[f'lag_{i+1}_interval'] = [last_known_intervals_series.iloc[N_LAGS - 1 - i]]

        input_df_for_prediction = pd.DataFrame(input_data_for_prediction, columns=features_for_model)

        print("\nUltimi intervalli osservati (usati come input per la predizione):")
        print(last_known_intervals_series)
        print("\nDataFrame di input per la predizione del prossimo intervallo:")
        print(input_df_for_prediction)

        predicted_next_interval = model.predict(input_df_for_prediction)
        print(f"\nPredizione per la lunghezza del prossimo intervallo tra pareggi: {predicted_next_interval[0]:.2f} partite")
    else:
        print("Non ci sono abbastanza dati storici recenti per fare una predizione del prossimo intervallo.")

    print("\n--- Fine Fase 6: Predizione del Prossimo Intervallo ---")
else:
    print("\n--- Fase 6: Predizione del Prossimo Intervallo SKIPPATA ---")


print("\n--- Fine dell'analisi. Ciao Andrea! ---")



Ciao Andrea! Iniziamo l'analisi per la predizione della distanza tra pareggi usando CatBoost.
Dati caricati con successo. Numero di righe: 30891, Numero di colonne: 7

--- Inizio Fase 1: Preparazione Dati Iniziale ---
Attenzione: Alcune date non sono state convertite correttamente in datetime.
         date      hour
0  15-05-2025  12:40:00
1  15-05-2025  12:34:00
2  15-05-2025  12:28:00
3  15-05-2025  12:22:00
4  15-05-2025  12:16:00
DataFrame vuoto dopo la rimozione di righe con datetime non validi.
Feature 'global_distance_since_last_draw' calcolata.
Empty DataFrame
Columns: [datetime, is_draw, global_distance_since_last_draw]
Index: []

--- Fine Fase 1: Preparazione Dati Iniziale ---

--- Inizio Fase 2: Estrazione Sequenza Intervalli tra Pareggi ---
Nessun evento di pareggio trovato nel dataset. Impossibile procedere.
Numero di eventi di pareggio identificati: 0
Primi eventi di pareggio e la lunghezza dell'intervallo corrispondente:
Empty DataFrame
Columns: [datetime, draw_interval

C:\Users\dyshkantiuk_andrii\AppData\Local\Temp\ipykernel_23252\4204359697.py:97: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['global_distance_since_last_draw'].fillna(0, inplace=True) # Per la prima partita, o se la precedente era un pareggio.
